# Merging a broken .csv


In [1]:
import pandas as pd
import os 
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np

np.set_printoptions(suppress=True)

#Decide what columns we need
A_csv = pd.read_csv('../xaa.csv')
B_csv = pd.read_csv('../xab.csv', header=None)
C_csv = pd.read_csv('../xac.csv', header=None)
D_csv = pd.read_csv('../xad.csv', header=None)
#Add correct index to split csv
B_csv.columns = A_csv.columns
C_csv.columns = A_csv.columns
D_csv.columns = A_csv.columns

#Merging
AB_csv = pd.merge(A_csv, B_csv,how='outer')

ABC_csv = pd.merge(AB_csv, C_csv,how='outer')

ABCD_csv = pd.merge(ABC_csv, D_csv,how='outer')

master_df = ABCD_csv

display(master_df.head(1))
       

,Year,StateAbbr,CityName,GeographicLevel,Category,Measure,DataValueTypeID,Data_Value_Type,Data_Value,PopulationCount,GeoLocation,CategoryID,MeasureId
0,2016,US,NaN,US,Prevention,Current lack of health insurance among adults ...,AgeAdjPrv,Age-adjusted prevalence,14.6,308745538,NaN,PREVENT,ACCESS2


In [2]:
#Rename col more logical for vis
master_df_rename = master_df.rename(columns = {"Data_Value": "Percent_of_pop"})

#For Syntax Reference <<<< only_billys = df.loc[df["first_name"] == "Billy", :] >>>>

# To remove all city sub tracts for mean by State aggregation 
master_df_re_adj = master_df_rename.loc[master_df_rename['DataValueTypeID']=='AgeAdjPrv',:]
# To remove Us data rows
master_df_re_adj_nous = master_df_re_adj.dropna(subset=['CityName', 'GeoLocation'])
#Drop Year(some data collected every two years)
master_df_re_adj_noyr = master_df_re_adj_nous.drop(columns = "Year")

new_master_df = master_df_re_adj_noyr
#Verifying no citeies were lost 
display(
    len(new_master_df['CityName'].unique()),
    new_master_df.shape,
    len(master_df['CityName'].unique()),
    master_df.shape,   
    len(master_df['Measure'].unique())
)

474

(14000, 12)

475

(810103, 13)

28

### NEW MASTER ###


In [7]:
#Create new master (munging done)
#Creating a column to account for the people indicated in each measure category
#Syntax reference
#df= pd.DataFrame(range(5), columns=['a'])
#df.a = df.a.astype(float)


new_master_df = new_master_df.assign(Measure_pop = master_df_re_adj_noyr['Percent_of_pop'] / 100 * master_df_re_adj_noyr['PopulationCount'])

new_master_df['Measure_pop'] = new_master_df['Measure_pop'].fillna(0).astype(int)

new_master_df.head(1)

#master_state_measure_gbdf = master_state_measure_gbdf.assign(true_st_percent = master_state_measure_gbdf['Measure_pop'] / master_state_measure_gbdf['PopulationCount']*100)


#master_state_measure_gbdf.drop(columns = 'Percent_of_pop') 

,StateAbbr,CityName,GeographicLevel,Category,Measure,DataValueTypeID,Data_Value_Type,Percent_of_pop,PopulationCount,GeoLocation,CategoryID,MeasureId,Measure_pop
56,AL,Birmingham,City,Prevention,Current lack of health insurance among adults ...,AgeAdjPrv,Age-adjusted prevalence,19.5,212237,"(33.5275663773, -86.7988174678)",PREVENT,ACCESS2,41386


### GROUPBY ###


In [4]:
###GROUPBY###

master_state_measure_gbdf = new_master_df.groupby(['Category','StateAbbr', 'Measure']).sum()
#For syntax reference
#(IGNORE)master_df_re_adj_noyr = master_df_re_adj_noyr.assign(Measure_pop = master_df_re_adj_noyr['Percent_of_pop'] / 100 * master_df_re_adj_noyr['PopulationCount'])
#Make True st percent and drop summed percent coln
master_state_measure_gbdf = master_state_measure_gbdf.assign(true_st_percent = master_state_measure_gbdf['Measure_pop'] / master_state_measure_gbdf['PopulationCount']*100)

master_state_measure_gbdf.drop(columns = 'Percent_of_pop') 





PopulationCount  \
Category            StateAbbr Measure                                                               
Health Outcomes     AK        All teeth lost among adults aged >=65 Years                  291826   
                              Arthritis among adults aged >=18 Years                       291826   
                              Cancer (excluding skin cancer) among adults age...           291826   
                              Chronic kidney disease among adults aged >=18 Y...           291826   
                              Chronic obstructive pulmonary disease among adu...           291826   
                              Coronary heart disease among adults aged >=18 Y...           291826   
                              Current asthma among adults aged >=18 Years                  291826   
                              Diagnosed diabetes among adults aged >=18 Years              291826   
                              High blood pressure among adults aged >=18 Years             291826   
                              High cholesterol among adults aged >=18 Years w...           291826   
                              Mental health not good for >=14 days among adul...           291826   
                              Physical health not good for >=14 days among ad...           291826   
                              Stroke among adults aged >=18 Years                          291826   
                    AL        All teeth lost among adults aged >=65 Years                  965304   
                              Arthritis among adults aged >=18 Years                       965304   
                              Cancer (excluding skin cancer) among adults age...           965304   
                              Chronic kidney disease among adults aged >=18 Y...           965304   
                              Chronic obstructive pulmonary disease among adu...           965304   
                              Coronary heart disease among adults aged >=18 Y...           965304   
                              Current asthma among adults aged >=18 Years                  965304   
                              Diagnosed diabetes among adults aged >=18 Years              965304   
                              High blood pressure among adults aged >=18 Years             965304   
                              High cholesterol among adults aged >=18 Years w...           965304   
                              Mental health not good for >=14 days among adul...           965304   
                              Physical health not good for >=14 days among ad...           965304   
                              Stroke among adults aged >=18 Years                          965304   
                    AR        All teeth lost among adults aged >=65 Years                  490373   
                              Arthritis among adults aged >=18 Years                       490373   
                              Cancer (excluding skin cancer) among adults age...           490373   
                              Chronic kidney disease among adults aged >=18 Y...           490373   
...                                                                                           ...   
Unhealthy Behaviors VA        Binge drinking among adults aged >=18 Years                 1918061   
                              Current smoking among adults aged >=18 Years                1918061   
                              No leisure-time physical activity among adults ...          1918061   
                              Obesity among adults aged >=18 Years                        1918061   
                              Sleeping less than 7 hours among adults aged >=...          1918061   
                    VT        Binge drinking among adults aged >=18 Years                   42417   
                              Current smoking among adults aged >=18 Years                  42417   
                              No leisure-t

In [8]:
#reset index
master_state_measure_df = master_state_measure_gbdf.reset_index()

master_state_measure_df.keys()

category_list = master_state_measure_df['Category'].unique()
state_list = master_state_measure_df['StateAbbr'].unique()
measure_list = master_state_measure_df['Measure'].unique()


display(
    master_state_measure_df.keys(),
    category_list,
    state_list,
    measure_list
)


Index(['Category', 'StateAbbr', 'Measure', 'Percent_of_pop', 'PopulationCount',
       'Measure_pop', 'true_st_percent'],
      dtype='object')

array(['Health Outcomes', 'Prevention', 'Unhealthy Behaviors'],
      dtype=object)

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

array(['All teeth lost among adults aged >=65 Years',
       'Arthritis among adults aged >=18 Years',
       'Cancer (excluding skin cancer) among adults aged >=18 Years',
       'Chronic kidney disease among adults aged >=18 Years',
       'Chronic obstructive pulmonary disease among adults aged >=18 Years',
       'Coronary heart disease among adults aged >=18 Years',
       'Current asthma among adults aged >=18 Years',
       'Diagnosed diabetes among adults aged >=18 Years',
       'High blood pressure among adults aged >=18 Years',
       'High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years',
       'Mental health not good for >=14 days among adults aged >=18 Years',
       'Physical health not good for >=14 days among adults aged >=18 Years',
       'Stroke among adults aged >=18 Years',
       'Cholesterol screening among adults aged >=18 Years',
       'Current lack of health insurance among adults aged 18–64 Years',
       'Fecal occult b

# FUNCTION #

In [11]:
###Function_Time###only_billys = df.loc[df["first_name"] == "Billy", :]

# df_list = []
# for category in category_list:
#         print(category)
#         if master_state_measure_df.iloc['Category'] == category:
#             df_list.append(master_state_measure_df['StateAbbr'])

# df_list

In [ ]:
####FUNCTION 2 
#df_healthy.reset_index()
#df_outcome.reset_index()
#df_unhealthy.reset_index()
####reset index 

# big_unstack = unhealthy_df.unstack()

# big_unstack.shape


In [12]:
healthy_measures = df_healthy['Measure'].unique()
outcome_measures = df_outcome['Measure'].unique()
unhealthy_measures = df_unhealthy['Measure'].unique()

NameError: name 'df_healthy' is not defined

In [ ]:
master_df.groupby(['StateAbbr', 'CityName']).count()

In [13]:
##Original Work with oversized CSV##
#cities_500_csv_dropped_colns = cities_500_csv
#cities_500_csv_dropped_colns = cities_500_csv_dropped_colns[['Year', 'StateAbbr', 'CityName', 'CategoryID','Measure', 'Data_Value','GeoLocation']]
#average the meaningful data ('Measure')
#Mean_cities_measures_df = cities_500_csv_dropped_colns.groupby(['Year','StateAbbr','Measure']).mean()
#Mean_cities_measures_df.head()
#############################################

#Breaking into "HLTHOUT" , PREVENT, and UNHBEH
all_HLTHOUT = master_df.loc[master_df['CategoryID'] == 'HLTHOUT', :]
all_PREVENT = master_df.loc[master_df['CategoryID'] == 'PREVENT', :]
all_UNHBEH = master_df.loc[master_df['CategoryID'] == 'UNHBEH', :]

display(
    all_HLTHOUT.head(),
    all_HLTHOUT['Measure'].unique(),
    all_PREVENT.head(),
    all_PREVENT['Measure'].unique(),
    all_UNHBEH.head(),
    all_UNHBEH['Measure'].unique()
)

,Year,StateAbbr,CityName,GeographicLevel,Category,Measure,DataValueTypeID,Data_Value_Type,Data_Value,PopulationCount,GeoLocation,CategoryID,MeasureId
2,2016,US,NaN,US,Health Outcomes,Arthritis among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,23.0,308745538,NaN,HLTHOUT,ARTHRITIS
3,2016,US,NaN,US,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,25.4,308745538,NaN,HLTHOUT,ARTHRITIS
6,2015,US,NaN,US,Health Outcomes,High blood pressure among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,29.4,308745538,NaN,HLTHOUT,BPHIGH
7,2015,US,NaN,US,Health Outcomes,High blood pressure among adults aged >=18 Years,CrdPrv,Crude prevalence,31.9,308745538,NaN,HLTHOUT,BPHIGH
10,2016,US,NaN,US,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,AgeAdjPrv,Age-adjusted prevalence,5.9,308745538,NaN,HLTHOUT,CANCER


array(['Arthritis among adults aged >=18 Years',
       'High blood pressure among adults aged >=18 Years',
       'Cancer (excluding skin cancer) among adults aged >=18 Years',
       'Current asthma among adults aged >=18 Years',
       'Coronary heart disease among adults aged >=18 Years',
       'Chronic obstructive pulmonary disease among adults aged >=18 Years',
       'Diagnosed diabetes among adults aged >=18 Years',
       'High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years',
       'Chronic kidney disease among adults aged >=18 Years',
       'Mental health not good for >=14 days among adults aged >=18 Years',
       'Physical health not good for >=14 days among adults aged >=18 Years',
       'Stroke among adults aged >=18 Years',
       'All teeth lost among adults aged >=65 Years'], dtype=object)

,Year,StateAbbr,CityName,GeographicLevel,Category,Measure,DataValueTypeID,Data_Value_Type,Data_Value,PopulationCount,GeoLocation,CategoryID,MeasureId
0,2016,US,NaN,US,Prevention,Current lack of health insurance among adults ...,AgeAdjPrv,Age-adjusted prevalence,14.6,308745538,NaN,PREVENT,ACCESS2
1,2016,US,NaN,US,Prevention,Current lack of health insurance among adults ...,CrdPrv,Crude prevalence,11.6,308745538,NaN,PREVENT,ACCESS2
8,2015,US,NaN,US,Prevention,Taking medicine for high blood pressure contro...,AgeAdjPrv,Age-adjusted prevalence,57.7,308745538,NaN,PREVENT,BPMED
9,2015,US,NaN,US,Prevention,Taking medicine for high blood pressure contro...,CrdPrv,Crude prevalence,77.2,308745538,NaN,PREVENT,BPMED
16,2016,US,NaN,US,Prevention,Visits to doctor for routine checkup within th...,AgeAdjPrv,Age-adjusted prevalence,69.7,308745538,NaN,PREVENT,CHECKUP


array(['Current lack of health insurance among adults aged 18–64 Years',
       'Taking medicine for high blood pressure control among adults aged >=18 Years with high blood pressure',
       'Visits to doctor for routine checkup within the past Year among adults aged >=18 Years',
       'Cholesterol screening among adults aged >=18 Years',
       'Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50–75 Years',
       'Papanicolaou smear use among adult women aged 21–65 Years',
       'Older adult men aged >=65 Years who are up to date on a core set of clinical preventive services: Flu shot past Year, PPV shot ever, Colorectal cancer screening',
       'Older adult women aged >=65 Years who are up to date on a core set of clinical preventive services: Flu shot past Year, PPV shot ever, Colorectal cancer screening, and Mammogram past 2 Years',
       'Visits to dentist or dental clinic among adults aged >=18 Years',
       'Mammography use among women aged 50–74 Y

,Year,StateAbbr,CityName,GeographicLevel,Category,Measure,DataValueTypeID,Data_Value_Type,Data_Value,PopulationCount,GeoLocation,CategoryID,MeasureId
4,2016,US,NaN,US,Unhealthy Behaviors,Binge drinking among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,18.0,308745538,NaN,UNHBEH,BINGE
5,2016,US,NaN,US,Unhealthy Behaviors,Binge drinking among adults aged >=18 Years,CrdPrv,Crude prevalence,16.9,308745538,NaN,UNHBEH,BINGE
29,2016,US,NaN,US,Unhealthy Behaviors,Current smoking among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,16.8,308745538,NaN,UNHBEH,CSMOKING
30,2016,US,NaN,US,Unhealthy Behaviors,Current smoking among adults aged >=18 Years,CrdPrv,Crude prevalence,16.4,308745538,NaN,UNHBEH,CSMOKING
39,2016,US,NaN,US,Unhealthy Behaviors,No leisure-time physical activity among adults...,AgeAdjPrv,Age-adjusted prevalence,23.7,308745538,NaN,UNHBEH,LPA


array(['Binge drinking among adults aged >=18 Years',
       'Current smoking among adults aged >=18 Years',
       'No leisure-time physical activity among adults aged >=18 Years',
       'Obesity among adults aged >=18 Years',
       'Sleeping less than 7 hours among adults aged >=18 Years'],
      dtype=object)

In [ ]:
unhealthy_means = all_UNHBEH.groupby(['StateAbbr','Measure']).mean()
healthy_means = all_PREVENT.groupby(['StateAbbr','Measure']).mean()
outcome_means = all_HLTHOUT.groupby(['StateAbbr','Measure']).mean()

display(
    healthy_means.head(),
    outcome_means.head(),
    unhealthy_means.head()
)

In [ ]:
#Reset index for .loc

unhealthy_means = unhealthy_means.reset_index()



In [ ]:
#Boolean measures
#only_billys = df.loc[df["first_name"] == "Billy", :]
unhealthy_means_binge= unhealthy_means.loc[unhealthy_means['Measure']=="Binge drinking among adults aged >=18 Years", :]


unhealthy_means_binge = unhealthy_means_binge.sort_values(by=['%_of_pop'],ascending=False)

unhealthy_means_binge["Rank"] = range(len(unhealthy_means_binge['StateAbbr']))#+1
                                                          
unhealthy_means_binge = unhealthy_means_binge.drop(columns = "level_0")
unhealthy_means_binge = unhealthy_means_binge.drop(columns = "index")
unhealthy_means_binge

In [ ]:
#Test Plots
# Plot
fig, (st_chol, st_bp, st_heart) = plt.subplots(3, sharex=True)
fig.suptitle("Crime Rates Over Time", fontsize=16, fontweight="bold")

ax1.set_xlim(min(year), max(year))
ax1.plot(year, violent_crime_rate, linewidth=1, marker="o")
ax1.plot(year, vc_fit, "b--", linewidth=1)
ax1.set_ylabel("Violent Crime Rate")


In [ ]:
#create df for healthy, unhealthy, and outcomes.

df_structure = pd.DataFrame ([
    {"State": 'TX', 'Measure': '%chol_screen>18', 'Rank': '16th'},
    {"State": 'CA', 'Measure': '%chol_screen>18', 'Rank': '3rd'},
    {"State": 'MI', 'Measure': '%chol_screen>18', 'Rank': '47th'}
    
])
df_structure

In [ ]:
#create df for healthy, unhealthy, and outcomes. Idea is to have them look like this
# Data frame title will denote the measure displayed in the table
df_structure = pd.DataFrame(
    {"State": ['TX', 'CA', 'MI'],
     "%_of_pop": ['75', '85', '59'],
     "Rank": ['7th', '3rd', '46th']}
)
df_structure



In [ ]:
## How to separate by healthy, unhealthy, and outcome.
#need to drop year col and group by state



In [ ]:
url = https://healthdata.gov/data.json


In [ ]:
## Create ranking system with cities and states